In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10


In [2]:
net_in = keras.layers.Input((32,32,3))
net_out = keras.layers.Conv2D(96, 7)(net_in)
net_out = keras.layers.Dense(10, activation='softmax')(net_out)
model = keras.Model(inputs=net_in, outputs=net_out)

In [9]:
L = 1e-2

net_in = keras.layers.Input((32,32,3))
net_out = keras.layers.Conv2D(96, 3, kernel_regularizer=keras.regularizers.l2(L))(net_in)
net_out = keras.layers.BatchNormalization()(net_out)
net_out = keras.layers.ReLU()(net_out)
net_out = keras.layers.Conv2D(96, 3, kernel_regularizer=keras.regularizers.l2(L))(net_out)
net_out = keras.layers.BatchNormalization()(net_out)
net_out = keras.layers.ReLU()(net_out)
net_out = keras.layers.Conv2D(96, 3, strides=2, kernel_regularizer=keras.regularizers.l2(L))(net_out)
net_out = keras.layers.BatchNormalization()(net_out)
net_out = keras.layers.ReLU()(net_out)
#net_out = keras.layers.MaxPool2D(3, strides=2)(net_out)
net_out = keras.layers.Conv2D(192, 3, kernel_regularizer=keras.regularizers.l2(L))(net_out)
net_out = keras.layers.BatchNormalization()(net_out)
net_out = keras.layers.ReLU()(net_out)
net_out = keras.layers.Conv2D(192, 3, kernel_regularizer=keras.regularizers.l2(L))(net_out)
net_out = keras.layers.BatchNormalization()(net_out)
net_out = keras.layers.ReLU()(net_out)
net_out = keras.layers.Conv2D(192, 3, strides=2, kernel_regularizer=keras.regularizers.l2(L))(net_out)
net_out = keras.layers.BatchNormalization()(net_out)
net_out = keras.layers.ReLU()(net_out)
#net_out = keras.layers.MaxPool2D(3, strides=2)(net_out)
net_out = keras.layers.Conv2D(192, 3, kernel_regularizer=keras.regularizers.l2(L))(net_out)
net_out = keras.layers.BatchNormalization()(net_out)
net_out = keras.layers.ReLU()(net_out)
net_out = keras.layers.GlobalAvgPool2D()(net_out)
net_out = keras.layers.Dense(192, activation='relu', kernel_regularizer=keras.regularizers.l2(L))(net_out)
net_out = keras.layers.Dense(10, activation='softmax', kernel_regularizer=keras.regularizers.l2(L))(net_out)

model = keras.Model(inputs=net_in, outputs=net_out)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 30, 30, 96)        2688      
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 30, 96)        384       
_________________________________________________________________
re_lu_7 (ReLU)               (None, 30, 30, 96)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 96)        83040     
_________________________________________________________________
batch_normalization_8 (Batch (None, 28, 28, 96)        384       
_________________________________________________________________
re_lu_8 (ReLU)               (None, 28, 28, 96)        0   

In [3]:
model = keras.models.load_model('../allconv_cifar10.h5')

In [10]:
model.load_weights('../allconv_cifar10.h5')

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
lr = 0.25

model.compile(optimizer=keras.optimizers.SGD(learning_rate=lambda: lr, momentum=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [9]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
lr = 0.0025
model.compile(optimizer=keras.optimizers.SGD(learning_rate=lambda: lr, momentum=0.9),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=25)

Train on 50000 samples
Epoch 1/25
50000/50000 [==============================] - 13s 262us/sample - loss: 4128.4852 - accuracy: 0.9549
Epoch 2/25
50000/50000 [==============================] - 12s 239us/sample - loss: 858.8340 - accuracy: 0.9575
Epoch 3/25
50000/50000 [==============================] - 12s 240us/sample - loss: 178.8553 - accuracy: 0.9619
Epoch 4/25
50000/50000 [==============================] - 12s 239us/sample - loss: 37.4256 - accuracy: 0.9622
Epoch 5/25
50000/50000 [==============================] - 12s 240us/sample - loss: 8.0868 - accuracy: 0.9530
Epoch 6/25
50000/50000 [==============================] - 12s 238us/sample - loss: 2.6574 - accuracy: 0.8347
Epoch 7/25
50000/50000 [==============================] - 12s 239us/sample - loss: 1.7749 - accuracy: 0.7619
Epoch 8/25
50000/50000 [==============================] - 12s 239us/sample - loss: 1.5293 - accuracy: 0.7422
Epoch 9/25
50000/50000 [==============================] - 12s 237us/sample - loss: 1.4303 - accur

In [13]:
from time import time

with tf.device('device:CPU:0'):
    start = time()
    model.evaluate(x_test, y_test, batch_size=32)
    stop = time()
    print(1e6 * (stop-start) / 10000)
    
with tf.device('device:GPU:0'):
    start = time()
    model.evaluate(x_test, y_test, batch_size=32)
    stop = time()
    print(1e6 * (stop-start) / 10000)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1219.1298246383667
10000/1 [============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

267.90432929992676


In [23]:
model.save('allconv_cifar10.h5')

In [3]:
tf.version.GIT_VERSION

'v2.2.0-rc4-8-g2b96f3662b'

In [6]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
class Conv2DLowRank(tf.keras.layers.Layer):
    def __init__(self, filters, rank, kernel_size, strides=(1,1), 
                 padding='valid', activation=None, use_bias=True, **kwargs):
        super(Conv2DLowRank, self).__init__(**kwargs)
        self.filters = filters
        self.rank = rank
        self.kernel_size = 2*(kernel_size,) if isinstance(kernel_size, int) else kernel_size
        self.strides = 2*(strides,) if isinstance(strides, int) else strides
        self.padding = padding
        self.activation = activation
        self.use_bias = use_bias
        v, h = self.kernel_size
        vs, hs = self.strides
        self.v_layer = tf.keras.layers.Conv2D(rank, kernel_size=(v,1), strides=(vs,1), 
                                              padding=padding, activation='linear', use_bias=False)
        self.h_layer = tf.keras.layers.Conv2D(filters, kernel_size=(1,h), strides=(1,hs), 
                                              padding=padding, activation=activation, use_bias=use_bias)

    def call(self, inputs):
        return self.h_layer(self.v_layer(inputs))
    
    def get_config(self):
        config = super(Conv2DLowRank, self).get_config()
        config.update({'filters':self.filters, 'rank':self.rank, 'kernel_size':self.kernel_size, 
                       'strides':self.strides, 'padding':self.padding, 'activation':self.activation, 
                       'use_bias':self.use_bias})
        return config


In [3]:
import numpy

def LR_clone(layer, ranks):
    if isinstance(layer, tf.keras.layers.Conv2D):
        w = layer.weights[0].numpy()
        C = w.shape[2]
        N = w.shape[3]
        v = w.shape[0]
        h = w.shape[1]
        if v == 1 or h == 1 or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            return layer.__class__.from_config(layer.get_config())
        config = layer.get_config()
        name = config['name']
        stride = config['strides']
        pad = config['padding']
        act = config['activation']
        bias = config['use_bias']
        K = ranks[layer.name] #min(C*v//4, h*N//4) + 3
        return Conv2DLowRank(N, K, (v,h), strides=stride, padding=pad, activation=act, use_bias=bias, name=name)
    else:
        return layer.__class__.from_config(layer.get_config())

def LR_optimize(model, ranks):
    clone = tf.keras.models.clone_model(model, clone_function=LR_clone)
    #out_layer = model.input
    for i in range(len(clone.layers)):
        layer = model.get_layer(index=i)
        clone_layer = clone.get_layer(index=i)
        if isinstance(clone_layer, Conv2DLowRank):
            w = layer.weights[0].numpy() # 4d tensor
            w = w.transpose((2, 0, 1, 3))
            C = w.shape[0]
            N = w.shape[3]
            d = w.shape[1]
            shape2d = (w.shape[0]*w.shape[1], w.shape[2]*w.shape[3]) # (Cd x dN)
            w2d = numpy.empty(shape2d, dtype='float32')
            assert w.shape[1] == w.shape[2]
            #d = w.shape[1]
            for i1 in range(C):
                for i2 in range(d):
                    for i3 in range(d):
                        for i4 in range(N):
                            w2d[i1*d+i2, i4*d+i3] = w[i1,i2,i3,i4]
            U, D, Q = numpy.linalg.svd(w2d)
            Q = Q.transpose()
            D = numpy.sqrt(D) # siempre son >0 los valores singulares??
            K = ranks[layer.name]
            V = numpy.empty((d,1,C,K), dtype='float32')
            H = numpy.empty((1,d,K,N), dtype='float32')
            for k in range(K):
                for c in range(C):
                    V[:, 0, c, k] = U[c*d : c*d+d, k] * D[k]
            for n in range(N):
                for k in range(K):
                    H[0, :, k, n] = Q[n*d : n*d+d, k] * D[k]
            #act_func = layer.get_config()['activation']
            #V_layer = tf.keras.layers.Conv2D(K, kernel_size=(d,1), activation='linear', use_bias=False)
            #H_layer = tf.keras.layers.Conv2D(N, kernel_size=(1,d), activation=act_func)
            #out_layer = H_layer(V_layer(out_layer))
            clone_layer.v_layer.set_weights([V])
            if layer.get_config()['use_bias']:
                b = layer.weights[1].numpy()
                clone_layer.h_layer.set_weights([H,b])
            else:
                clone_layer.h_layer.set_weights([H])
        else:
            clone_layer.set_weights(layer.get_weights())

    return clone

In [4]:
import numpy # NUEVO

def LR_clone(layer, weight_dict, ratio=0.95): # automatic rank choosing
    assert ratio == 'manual' or ratio < 1
    if isinstance(layer, tf.keras.layers.Conv2D):
        w = layer.weights[0].numpy()
        w = w.transpose((2, 0, 1, 3))
        C = w.shape[0]
        N = w.shape[3]
        d = w.shape[1]
        if isinstance(layer, tf.keras.layers.DepthwiseConv2D): # or 1 in w.shape
            return layer.__class__.from_config(layer.get_config())
        assert w.shape[1] == w.shape[2]
        shape2d = (w.shape[0]*w.shape[1], w.shape[2]*w.shape[3]) # (Cd x dN)
        w2d = numpy.empty(shape2d, dtype='float32')
        for i1 in range(C):
            for i2 in range(d):
                for i3 in range(d):
                    for i4 in range(N):
                        w2d[i1*d+i2, i4*d+i3] = w[i1,i2,i3,i4]
        U, D, Q = numpy.linalg.svd(w2d)
        Q = Q.transpose()
        
        if ratio == 'manual':
            K = weight_dict[layer.name]
        else:
            minVar = ratio * D.sum() # 0.95
            K = 1 
            for i in D.cumsum(): #TODO: busqueda binaria
                if i > minVar:
                    break
                K += 1        
        if K >= d*C*N/(C+N): # K < dCN/(C+N) para acelerar
            return layer.__class__.from_config(layer.get_config())
        
        D = numpy.sqrt(D)          
        V = numpy.empty((d,1,C,K), dtype='float32')
        H = numpy.empty((1,d,K,N), dtype='float32')
        for k in range(K):
            for c in range(C):
                V[:, 0, c, k] = U[c*d : c*d+d, k] * D[k]
        for n in range(N):
            for k in range(K):
                H[0, :, k, n] = Q[n*d : n*d+d, k] * D[k]        
                
        config = layer.get_config()
        name = config['name']
        stride = config['strides']
        pad = config['padding']
        act = config['activation']
        bias = config['use_bias']
        newlayer = Conv2DLowRank(N, K, d, strides=stride, padding=pad, 
                                 activation=act, use_bias=bias, name=name)
        weight_dict[newlayer.name] = [V, H]
        if bias: weight_dict[newlayer.name].append(layer.weights[1].numpy())
        return newlayer
    else:
        return layer.__class__.from_config(layer.get_config())

def LR_optimize(model, ratio=0.95, LR_weights=dict()):
    #LR_weights = dict()
    clone = tf.keras.models.clone_model(model, clone_function=lambda l: LR_clone(l, LR_weights, ratio))
    for i in range(len(clone.layers)):
        layer = model.get_layer(index=i)
        clone_layer = clone.get_layer(index=i)
        if isinstance(clone_layer, Conv2DLowRank):
            weights = LR_weights[clone_layer.name] # [V,H,b]
            clone_layer.v_layer.set_weights(weights[:1])
            clone_layer.h_layer.set_weights(weights[1:])
        else:
            clone_layer.set_weights(layer.get_weights())

    return clone

In [7]:
#model = tf.keras.models.load_model('../models/VGG16bn.h5', compile=False)
model = tf.keras.models.load_model('../lowrank/VGG16bn_lro/', compile=False)

In [8]:
model = model.layers[-1]

In [6]:
#losses = {'block_13_project': 0.898777, 'block_5_expand': 0.415501, 'block_13_expand': 0.857697, 'block_6_project': 0.8972786273807287, 'block_7_expand': 0.8827925343066454, 'block_8_expand': 0.5607217252254486, 'block_16_expand': 0.8765185568481684, 'block_12_expand': 0.8193134739995003, 'block_4_project': 0.7647058963775635, 'block_9_project': 0.3996163606643677, 'block_3_project': 0.8971787230111659, 'block_6_expand': 0.5945892035961151, 'block_10_expand': 0.5996443629264832, 'block_3_expand': 0.7847266793251038, 'block_5_project': 0.356317937374115, 'Conv1': 0.09235137701034546, 'block_2_expand': 0.8828524760901928, 'block_12_project': 0.8843710180372, 'block_2_project': 0.8937020609155297, 'block_14_project': 0.8958200272172689, 'block_16_project': 0.8960398165509105, 'block_10_project': 0.893502252176404, 'block_15_project': 0.8926430772989988, 'block_1_expand': 0.4621363580226898, 'block_7_project': 0.7913802862167358, 'expanded_conv_project': 0.898097840603441, 'block_11_expand': 0.8746003974229097, 'Conv_1': 0.8895460506901145, 'block_9_expand': 0.324248731136322, 'block_4_expand': 0.8682664781808853, 'block_14_expand': 0.8978980323299766, 'block_15_expand': 0.8879475854337215, 'block_11_project': 0.8816935885697603, 'block_1_project': 0.8962396243587136, 'block_8_project': 0.4169996976852417}
#dmobnet {'predictions': 0.880595}
#losses = {'conv5_block1_1_conv': 0.409035, 'conv1_conv': 0.004223477363586414, 'conv4_block6_3_conv': 0.28155721378326415, 'conv4_block6_2_conv': 0.3194208855628967, 'conv3_block4_1_conv': 0.008379470825195301, 'conv4_block4_1_conv': 0.19028485965728759, 'conv3_block4_3_conv': 0.009438466548919666, 'conv2_block3_2_conv': -0.020652640342712414, 'conv4_block1_3_conv': 0.1938614363670349, 'conv4_block4_3_conv': 0.17547906589508055, 'conv4_block2_1_conv': 0.1751793737411499, 'conv2_block2_1_conv': -0.020732570171356213, 'conv3_block1_1_conv': -0.033020782947540295, 'conv3_block1_2_conv': -0.016316819667816174, 'conv3_block3_2_conv': -0.006446290493011486, 'conv2_block1_1_conv': -0.004508126258850109, 'conv3_block3_1_conv': -0.010282624244689953, 'conv2_block2_2_conv': -0.01823495674133302, 'conv2_block3_3_conv': -0.02902458953857423, 'conv3_block2_1_conv': 0.009278606891632069, 'conv5_block1_0_conv': 0.6709034557342529, 'conv5_block2_2_conv': 0.6786160434484482, 'conv4_block3_3_conv': 0.13619674396514891, 'conv4_block3_2_conv': 0.1776369924545288, 'conv3_block1_0_conv': 0.04926021051406859, 'conv3_block4_2_conv': 0.030977797031402576, 'conv2_block1_0_conv': -0.00027220296859742366, 'conv5_block3_3_conv': 0.7747437621355057, 'conv4_block5_2_conv': 0.2172989363670349, 'conv4_block4_2_conv': 0.2060297245979309, 'conv3_block3_3_conv': -0.016576576709747326, 'conv4_block6_1_conv': 0.2759426350593567, 'conv5_block1_3_conv': 0.6561975593566894, 'conv5_block3_2_conv': 0.7876513714790344, 'conv2_block1_2_conv': -0.004328299045562756, 'conv4_block5_1_conv': 0.2371198649406433, 'conv4_block1_2_conv': 0.05385584783554076, 'conv2_block3_1_conv': -0.03180192756652833, 'conv2_block2_3_conv': -0.026147354125976574, 'conv5_block3_1_conv': 0.7770615483522415, 'conv4_block2_3_conv': 0.1485648865699768, 'conv5_block2_3_conv': 0.6573364849090576, 'conv4_block3_1_conv': 0.17196245145797728, 'conv4_block1_0_conv': 0.2273292655944824, 'conv5_block1_2_conv': 0.4568289632797241, 'conv5_block2_1_conv': 0.6986568088531494, 'conv3_block1_3_conv': 0.023245226860046375, 'conv2_block1_3_conv': -0.008464324951171887, 'conv3_block2_3_conv': 0.0014860744476318244, 'conv4_block5_3_conv': 0.18253226232528685, 'conv3_block2_2_conv': 0.017870378017425526, 'conv4_block2_2_conv': 0.19238282394409179, 'conv4_block1_1_conv': 0.024723779678344715}
#dres = {'predictions': 0.8903128209412098}
losses = {'block1_conv1': -0.018951, 'block4_conv2': 0.606949, 'block4_conv1': 0.49485304293632504, 'block3_conv1': 0.10637735662460324, 'block2_conv2': 0.040078573799133266, 'block5_conv1': 0.7409071558475494, 'block5_conv2': 0.8152352580189705, 'block1_conv2': 0.0199649864196777, 'block3_conv2': 0.19688838062286373, 'block5_conv3': 0.8360919216036796, 'block3_conv3': 0.22423964080810543, 'block2_conv1': 0.026242964839935268, 'block4_conv3': 0.6989938550949096}
#dvgg = {'predictions': 0.872506, 'fc2': 0.868267, 'fc1': 0.874926}

In [9]:
ranks = dict()
laynames = []
for layer in model.layers:
    if isinstance(layer, (tf.keras.layers.Conv2D)):
        #kernel_shape = layer.weights[0].shape #TODO detectar si capa es mas rapida o no (?)
        if isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            pass
        else: #isinstance(layer_ranks, int):
            #ranks[layer.name] = 5
            laynames.append(layer.name)

In [10]:
avg = 200
total_ranks = avg * len(losses)
total_loss = sum(losses.values())
#def_ranks = dict.fromkeys(laynames)
for i in range(len(laynames)):
    name = laynames[i] 
    if name in losses:
        curr_rank = max(int(avg * len(losses) * losses[name] / total_loss), 5)
        ranks[name] = curr_rank
        total_ranks -= curr_rank     
while total_ranks > 0:
    maxscore = 0
    for i in losses.keys():
        score = losses[i] / (2 * ranks[i] + 1)
        if maxscore < score:
            maxscore = score
            max_i = i
    ranks[max_i] += 1
    total_ranks -= 1

In [11]:
sum(ranks.values())/(avg*len(losses))

1.003076923076923

In [39]:
model.layers[10].output
# en Tai: C x d x d x N -> Cd x dN ... (i1,i2,i3,i4) -> (i1*d + i2, i4*d + i3)
# en Keras: d x d x C x N --- 2, 0, 1, 3 <-> 1, 2, 0, 3 

<tf.Tensor 'conv2d_3_1/Identity:0' shape=(None, 11, 11, 192) dtype=float32>

In [7]:
(_, _), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [5]:
#model = keras.models.load_model('vgg16/VGG16bn_70', compile=False)
#model = model.get_layer(index=-1)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
for i in model.layers:
    d = i.get_config()
    if 'rank' in d:
        print(d['rank'], end=', ')
#4 22 50 95 124 215 240 283 444 472 448 402 380 old vgg
#5, 8, 10, 16, 43, 80, 91, 201, 247, 284, 301, 332, 340, new vgg
#5, 10, 14, 21, 57, 106, 121, 268, 329, 379, 402, 442, 454,  x200
#5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 38, 18, 7, 14, 5, 5, 5, 5, 6, 24, 7, 19, 42, 179, 153, 138, 152, 117, 135, 140, 107, 150, 162, 138, 187, 171, 144, 252, 323, 360, 530, 536, 622, 
#5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 25, 12, 5, 9, 5, 5, 5, 5, 5, 16, 5, 13, 28, 119, 102, 92, 101, 78, 90, 93, 71, 100, 108, 92, 124, 114, 96, 145, 168, 148, 215, 240, 353, 345, 368, 357, 346, 409, 414, 408,
#181, 180 mobnet x150
#44, 54, 121, 121, 120, 119, 120, 118, 120, 120, mobnet x100
#239 mobnet x200

5, 24, 48, 48, 64, 128, 160, 192, 192, 256, 320, 320, 320, 

In [12]:
ranks

{'block1_conv1': 5,
 'block1_conv2': 10,
 'block2_conv1': 14,
 'block2_conv2': 21,
 'block3_conv1': 57,
 'block3_conv2': 106,
 'block3_conv3': 121,
 'block4_conv1': 268,
 'block4_conv2': 329,
 'block4_conv3': 379,
 'block5_conv1': 402,
 'block5_conv2': 442,
 'block5_conv3': 454}

In [4]:
model = tf.keras.applications.VGG16(weights='imagenet')

In [7]:
ranks = {'block1_conv1':5, 'block1_conv2':24, 'block2_conv1':48, 'block2_conv2':48, 
         'block3_conv1':64, 'block3_conv2':128, 'block3_conv3':160, 'block4_conv1':192, 
         'block4_conv2':192, 'block4_conv3':256, 'block5_conv1':320, 'block5_conv2':320, 
         'block5_conv3':320}

In [13]:
model_lr.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=['accuracy'])

In [21]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.4612 - accuracy: 0.7895


[1.4611903429031372, 0.7894999980926514]

In [8]:
#import pdb
#pdb.set_trace()
model_lr = LR_optimize(model, 'manual', ranks)
#vgg_lr = vgg_lr.layers[-1]

In [9]:
model_lr.summary()

Model: "vgg16_bn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2DLowRank) (None, 224, 224, 64)      1069      
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 224, 224, 64)      256       
_________________________________________________________________
block1_relu1 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2DLowRank) (None, 224, 224, 64)      9280      
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 224, 224, 64)      256       
_________________________________________________________________
block1_relu2 (Activation)    (None, 224, 224, 64)      0  

In [15]:
model_lr.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=['accuracy'])

In [10]:
model_lr.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.4625 - accuracy: 0.7897


[1.4624511003494263, 0.7896999716758728]

In [10]:
netin = keras.Input((224,224,3), dtype='uint8')
x = tf.cast(netin, 'float32')
#x = keras.applications.mobilenet_v2.preprocess_input(x) # !!!
#x = tf.nn.bias_add(x, [-123.68, -116.779, -103.939])
x = keras.applications.imagenet_utils.preprocess_input(x, mode='torch')
x = model_lr(x)
model_lr = keras.Model(inputs=netin, outputs=x)

In [ ]:
model_lr.save('VGG16bn_r200')

In [26]:
model_lr.layers[4].layers[1].get_config()

{'activation': 'relu',
 'dtype': 'float32',
 'filters': 64,
 'kernel_size': (3, 3),
 'name': 'conv2d_low_rank_7',
 'padding': 'same',
 'rank': 5,
 'strides': (1, 1),
 'trainable': True,
 'use_bias': True}

In [1]:
model.layers[1].get_config()

NameError: name 'model' is not defined

In [6]:
x_ds = tf.data.Dataset.from_tensor_slices(x_test.astype('float32'))
y_ds = tf.data.Dataset.from_tensor_slices(y_test.astype('float32'))

Time: 9.558 s


In [10]:
from time import time

b_sz = 2
#test_ds = tf.data.Dataset.zip((x_ds, y_ds))
test_ds = x_ds.take(1024).batch(b_sz)
steps = 1024 // b_sz
#batch = x_test[:b_sz,...]
start = time()
for batch in test_ds:
    prediction = model.predict(batch)
stop = time()
print('Time: %.3f s' % (stop - start))
print('Speed: %.1f inf/s' % (1024 / (stop-start)))

Time: 6.011 s
Speed: 170.3 inf/s


In [12]:
profiler.start()
mod_file = '../allconv_cifar10.h5'
b_sz  = 16
model = tf.keras.models.load_model(mod_file, compile=False)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_ds = tf.data.Dataset.from_tensor_slices(x_test.astype('float32'))
y_ds = tf.data.Dataset.from_tensor_slices(y_test.astype('float32'))

print('Evaluating accuracy...')
#_, acc = model.evaluate(x_test, y_test, batch_size=b_sz, verbose=0)

test_ds = x_ds.take(1024).batch(b_sz)
steps = 1024 // b_sz
print('Measuring speed...')
start = time()
for batch in test_ds:
    prediction = model.predict(batch)
stop = time()
prof_result = profiler.stop()
profiler.save('prof_log', prof_result)

print('Metrics for model %s, with batch size %d:' % (mod_file, b_sz))
print('Time: %.3f s' % (stop - start))
print('Speed: %.1f inf/s' % (1024 / (stop-start)))
#print('Accuracy: %.2f %', % 100 * acc)

Evaluating accuracy...
Measuring speed...
Metrics for model ../allconv_cifar10.h5, with batch size 16:
Time: 0.608 s
Speed: 1685.2 inf/s


B_SZ, TIME, SPEED
1, 354.706, 28.192
2, 178.401, 56.054
4, 91.185, 109.667
8, 42.331, 236.234

In [11]:
a = profiler.stop()

In [21]:
netin = tf.keras.Input((224,224,3))
net = tf.keras.layers.Conv2D(4, kernel_size=(3,1), padding='same')(netin)
net = tf.keras.layers.Conv2D(64, kernel_size=(1,3), padding='same', name='block1_conv1')(net)
net = tf.keras.layers.BatchNormalization(name='block1_bn1', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block1_relu1')(net)
net = tf.keras.layers.Conv2D(22, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(64, kernel_size=(1,3), padding='same', name='block1_conv2')(net)
net = tf.keras.layers.BatchNormalization(name='block1_bn2', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block1_relu2')(net)
net = tf.keras.layers.MaxPool2D(pool_size=2, name='block1_pool')(net)
net = tf.keras.layers.Conv2D(50, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(128, kernel_size=(1,3), padding='same', name='block2_conv1')(net)
net = tf.keras.layers.BatchNormalization(name='block2_bn1', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block2_relu1')(net)
net = tf.keras.layers.Conv2D(95, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(128, kernel_size=(1,3), padding='same', name='block2_conv2')(net)
net = tf.keras.layers.BatchNormalization(name='block2_bn2', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block2_relu2')(net)
net = tf.keras.layers.MaxPool2D(pool_size=2, name='block2_pool')(net)
net = tf.keras.layers.Conv2D(124, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(256, kernel_size=(1,3), padding='same', name='block3_conv1')(net)
net = tf.keras.layers.BatchNormalization(name='block3_bn1', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block3_relu1')(net)
net = tf.keras.layers.Conv2D(215, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(256, kernel_size=(1,3), padding='same', name='block3_conv2')(net)
net = tf.keras.layers.BatchNormalization(name='block3_bn2', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block3_relu2')(net)
net = tf.keras.layers.Conv2D(240, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(256, kernel_size=(1,3), padding='same', name='block3_conv3')(net)
net = tf.keras.layers.BatchNormalization(name='block3_bn3', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block3_relu3')(net)
net = tf.keras.layers.MaxPool2D(pool_size=2, name='block3_pool')(net)
net = tf.keras.layers.Conv2D(283, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(512, kernel_size=(1,3), padding='same', name='block4_conv1')(net)
net = tf.keras.layers.BatchNormalization(name='block4_bn1', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block4_relu1')(net)
net = tf.keras.layers.Conv2D(444, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(512, kernel_size=(1,3), padding='same', name='block4_conv2')(net)
net = tf.keras.layers.BatchNormalization(name='block4_bn2', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block4_relu2')(net)
net = tf.keras.layers.Conv2D(472, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(512, kernel_size=(1,3), padding='same', name='block4_conv3')(net)
net = tf.keras.layers.BatchNormalization(name='block4_bn3', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block4_relu3')(net)
net = tf.keras.layers.MaxPool2D(pool_size=2, name='block4_pool')(net)
net = tf.keras.layers.Conv2D(448, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(512, kernel_size=(1,3), padding='same', name='block5_conv1')(net)
net = tf.keras.layers.BatchNormalization(name='block5_bn1', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block5_relu1')(net)
net = tf.keras.layers.Conv2D(402, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(512, kernel_size=(1,3), padding='same', name='block5_conv2')(net)
net = tf.keras.layers.BatchNormalization(name='block5_bn2', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block5_relu2')(net)
net = tf.keras.layers.Conv2D(380, kernel_size=(3,1), padding='same')(net)
net = tf.keras.layers.Conv2D(512, kernel_size=(1,3), padding='same', name='block5_conv3')(net)
net = tf.keras.layers.BatchNormalization(name='block5_bn3', epsilon=1e-5)(net)
net = tf.keras.layers.Activation('relu', name='block5_relu3')(net)
net = tf.keras.layers.MaxPool2D(pool_size=2, name='block5_pool')(net)
net = tf.keras.layers.Flatten()(net)
net = tf.keras.layers.Dense(4096, activation='relu', name='fc1')(net)
net = tf.keras.layers.Dropout(0.5, name='drop1')(net)
net = tf.keras.layers.Dense(4096, activation='relu', name='fc2')(net)
net = tf.keras.layers.Dropout(0.5, name='drop2')(net)
net = tf.keras.layers.Dense(1000, activation='softmax', name='predictions')(net)

mymodelbn = tf.keras.Model(inputs=netin, outputs=net)


In [23]:
netin2 = keras.Input((224,224,3), dtype='uint8')
x = tf.cast(netin2, 'float32')
#x = keras.applications.mobilenet_v2.preprocess_input(x) # !!!
#x = tf.nn.bias_add(x, [-123.68, -116.779, -103.939])
x = keras.applications.imagenet_utils.preprocess_input(x, mode='torch')
x = mymodelbn(x)
mymodel = keras.Model(inputs=netin2, outputs=x)
mymodel.save('VGG16_70X')

W0128 02:34:45.263262 140071626897152 deprecation.py:506] From /home/alexis/tensorflow/lib/python3.5/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
model